In [37]:
import pandas as pd
import numpy as np
import random
import time

In [38]:
#df = pd.read_csv (r'C:\Notebook scripts\Features_Variant_1.csv')
df = pd.read_csv ('Features_Variant_1.csv')

data_train=np.asarray(df.iloc[:,:])
#print(np.shape(data_train))

#shuffle before folding
np.random.shuffle(data_train)

In [39]:
#fold_1, fold_2, fold_3, fold_4, fold_5= np.array_split(data_train,5)
data_train_arr=np.array_split(data_train,5) #{[],[],[],[],[]}

y_train_arr=[]
x_train_arr=[]
for i in range(len(data_train_arr)):
    y_train_arr.append(data_train_arr[i][:,-1].reshape((-1, 1)) )
    x_train_arr.append(data_train_arr[i][:,:-1])


In [40]:
def normalization(X):
    mean=np.mean(X,axis=0)#(1,53)
    std=np.std(X,axis=0)
    np.place(std, std==0, 1)#if std==0 then change it to 1 for division #std_item[std_item==0]=1 
    x_normalized=(X-mean)/std
    return x_normalized,mean,std
def normalization_with_params(X,mean,std):
    return (X-mean)/std

In [41]:
def linear_regression_predict(X,omega,omega0):
    prod=X.dot(omega)#(8190,1)
    #print(np.shape(y+prod))
    return prod+omega0

def root_mean_square_error(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    summands=np.power(y_actual-y_predict,2)
    #print(np.shape(summands))
    mse=np.sum(summands)/len(y_predict)
    return np.sqrt(mse)
   
def r_squared(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    nom=np.sum(np.power(y_actual-y_predict,2))
    denom=np.sum(np.power(y_actual-np.mean(y_actual,axis=0),2))
    return 1-nom/denom

In [42]:
def batch_gradient_descent(X,y_train,learning_rate,num_iterations):
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
    # init omega
    #cost = [0.0 for i in range(num_iterations)]
    it=0
    for it in range(num_iterations): 
        N=len(X)
        error=y_train-linear_regression_predict(X,omega,omega0)
        omega=omega+learning_rate*2*((X.T).dot(error))/N
        omega0=omega0+learning_rate*2*np.sum(error)/N
        cost[it] = r_squared(X,omega,omega0,y_train)
    return omega,omega0,cost

In [43]:
# function to create a list containing mini-batches 
def create_mini_batch(data,i, batch_size): 
    mini_batch = data[i * batch_size:(i + 1)*batch_size, :] 
    X_mini = mini_batch[:, :-1] 
    Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        
    if data.shape[0] % batch_size != 0: 
        mini_batch = data[i * batch_size:data.shape[0]] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
       
    return X_mini,Y_mini

def mini_batch_gradient_descent(X,y_train,learning_rate,num_epoches,batch_size):
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    #cost = [] 
    r2_tr=[]
    rmse_tr=[]
   
    it=0
    for it in range(num_epoches): 
        data = np.hstack((X, y_train)) 
        np.random.shuffle(data) #shuffle at each epoch (not every batch creation)
        n_minibatches = X.shape[0] // batch_size 
        i = 0
        for i in range(n_minibatches + 1):
            X_mini,y_mini= create_mini_batch(data,i, batch_size) 
        
            N=len(X_mini)
            error=y_mini-linear_regression_predict(X_mini,omega,omega0)
            omega=omega+learning_rate*2*((X_mini.T).dot(error))/N
            omega0=omega0+learning_rate*2*np.sum(error)/N
            r2_tr.append(r_squared(X_mini,omega,omega0, y_mini))
            rmse_tr.append(root_mean_square_error(X_mini,omega,omega0, y_mini))
    return omega,omega0,r2_tr,rmse_tr

In [44]:
r2=[]
rmse=[]
omega=[]
omega0=[]
r2_train=[]
rmse_train=[]
for item in range(len(x_train_arr)):
    x_test_rd=x_train_arr[item]
    y_test=y_train_arr[item]
    x_train_folds_arr=np.delete(x_train_arr,item)
    y_train_folds_arr=np.delete(y_train_arr,item)
    
    x_train_rd=np.empty((0,x_train_arr[item].shape[1]), int)
    y_train=np.empty((0,y_train_arr[item].shape[1]), int)
    #print(np.shape(y_train_folds_arr))
    for i in range (len(x_train_folds_arr)):
        #print(np.shape(y_train_folds_arr[i]))
        #x_train_all55 =(np.vstack((x_train_all55,x_train_folds_arr[i])))
        x_train_rd=np.concatenate((x_train_rd,x_train_arr[i]))
        y_train=np.concatenate((y_train,y_train_arr[i]))
    #print(np.shape(y_train))
    x_train,mean,std=normalization(x_train_rd)     #normalize all train dataset
    x_test=normalization_with_params(x_test_rd,mean,std)  #normalize test dataset with mean and std of train
    #print(x)
    #start = time.time()
    omega_item,omega0_item,r2_tr,rmse_tr=mini_batch_gradient_descent(x_train,y_train,learning_rate = 0.001,num_epoches=18,
                                                            batch_size=800)
    #stop = time.time()
    #duration = stop-start
    #print(duration)
    r2.insert(item,r_squared(x_test,omega_item,omega0_item,y_test))
    rmse.insert(item,root_mean_square_error(x_test,omega_item,omega0_item,y_test))
    omega.insert(item,omega_item)
    omega0.insert(item,omega0_item)
    r2_train.insert(item,r2_tr[-1])
    rmse_train.insert(item,rmse_tr[-1])
    #print(cost)
    print(r2[item],rmse[item],r2_train[item],rmse_train[item])

0.35935736135117347 25.510014679888798 0.3181766087435972 15.393699678983934
0.27166478740114497 33.57919112179417 0.2742744680107848 23.901812323680073
0.30317133377979766 28.881401751398286 0.45937139456617426 12.628048049960968
0.3121991177767124 29.91927924047553 0.32064459476198637 30.238922890364062
0.22622756624479956 30.9282223930979 0.3621144488688184 33.38607222710823


In [48]:

# initialise data of lists. 
data = {'1':[rmse[0],r2[0],rmse_train[0],r2_train[0],omega0[0]], 
        '2':[rmse[1],r2[1],rmse_train[0],r2_train[1],omega0[1]],
        '3':[rmse[2],r2[2],rmse_train[2],r2_train[2],omega0[2]],
        '4':[rmse[3],r2[3],rmse_train[3],r2_train[3],omega0[3]],
        '5':[rmse[4],r2[4],rmse_train[4],r2_train[4],omega0[4]],
        'E':[np.mean(rmse),np.mean(r2),np.mean(rmse_train),np.mean(r2_train),np.mean(omega0)],
        'SD':[np.std(rmse),np.std(r2),np.std(rmse_train),np.std(r2_train), np.std(omega0)]} 
 
# Creates pandas DataFrame. 
df1 = pd.DataFrame(data, index =['RMSE', 'R^2','RMSE train','R^2 train','omega0']) 
df2 = pd.DataFrame(np.concatenate((np.hstack(omega),np.mean(omega,axis=0),np.std(omega,axis=0)),axis=1),
                   columns=['1', '2', '3','4','5','E','SD'])
df =pd.concat([df1, df2], axis=0)
df

,1,2,3,4,5,E,SD
RMSE,25.510015,33.579191,28.881402,29.919279,30.928222,29.763622,2.638582
R^2,0.359357,0.271665,0.303171,0.312199,0.226228,0.294524,0.044221
RMSE train,15.393700,15.393700,12.628048,30.238923,33.386072,23.109711,8.080331
R^2 train,0.318177,0.274274,0.459371,0.320645,0.362114,0.346916,0.062722
omega0,5.597443,5.548323,5.597575,5.594833,5.603977,5.588430,0.020279
0,0.290804,-0.119145,0.131080,0.182099,0.146845,0.126337,0.134806
1,-0.390339,-0.509221,-0.506154,-0.543579,-0.492124,-0.488283,0.051813
2,-0.649807,-0.711850,-1.056927,-0.876925,-0.582959,-0.775694,0.171120
3,0.163863,0.109704,0.278731,0.065489,0.235286,0.170614,0.078331
4,-0.454946,0.264260,-0.221863,-0.620795,0.068629,-0.192943,0.325575
